PREREQUISTE

1. Create Snowflake user
Create User and update the credential as described here:
https://docs.snowflake.com/en/developer-guide/snowflake-cli-v2/connecting/specify-credentials

use role sysadmin and makesure you grant following

GRANT EXECUTE TASK ON ACCOUNT TO ROLE SYSADMIN;

2. Create SnowCLI utility - for authentication only
https://docs.snowflake.com/en/developer-guide/snowflake-cli-v2/installation/installation

3. Create Python environment


conda create --name snow_env --override-channels -c https://repo.anaconda.com/pkgs/snowflake python=3.10 numpy pandas pyarrow

conda activate snow_env


conda install snowflake-snowpark-python


pip install snowflake - U


pip install ploty

4. run in Visual code or your choice of notebook

In [ ]:
#Snowflake python API packages

from snowflake.core import Root
from snowflake.core.database import Database
from snowflake.core.schema import Schema
from snowflake.core.table import Table, TableColumn, PrimaryKey
from snowflake.core.warehouse import Warehouse
from snowflake.core.role import Role
from snowflake.core.grant import Grant
from snowflake.core.grant import Grant, Grantees, Privileges, Securables
from snowflake.core.stage import Stage
from snowflake.core.stage import StageCollection
from snowflake.core.task import StoredProcedureCall, Task
from snowflake.core.task.dagv1 import DAGOperation, DAG, DAGTask

from snowflake.snowpark import Session
from snowflake.snowpark.types import IntegerType, StringType, StructType, FloatType, StructField, DateType, Variant
from snowflake.snowpark.functions import udf, sum, col,array_construct,month,year,call_udf,lit
from snowflake.snowpark.version import VERSION
import snowflake.snowpark.functions as F
import snowflake.snowpark.types as T
from snowflake.snowpark.functions import sproc
#from snowflake.snowpark import Window

# Import Python packages
import pandas as pd
import plotly.express as px
import plotly.io as pio
from datetime import timedelta

In [ ]:
from snowflake.connector.config_manager import CONFIG_MANAGER
# file from connection is created
print(CONFIG_MANAGER.file_path)
# https://docs.snowflake.com/en/developer-guide/snowflake-cli-v2/connecting/specify-credentials
# location in mac ~/Library/Application Support/snowflake/config.toml
# following take default connection from above file
#session = Session.builder.getOrCreate()
# following for specific connection
session=Session.builder.config("connection_name","demodc").create()
snowpark_version = VERSION
# Current Environment Details
print('Account                     : {}'.format(session.get_current_account()))
print('User                        : {}'.format(session.get_current_user()))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))

root = Root(session) 

In [ ]:
# create database
tb101_db = root.databases.create(Database(name="tb_101"),  mode="ifNotExists") # orReplace, errorIfExists 


In [ ]:
#create schemas
raw_pos_sch = tb101_db.schemas.create(Schema(name="raw_pos"), mode="orReplace")
raw_cust_sch = tb101_db.schemas.create(Schema(name="raw_customer"), mode="orReplace")
harmonized_sch = tb101_db.schemas.create(Schema(name="harmonized"), mode="orReplace")
analytics_sch = tb101_db.schemas.create(Schema(name="analytics"), mode="orReplace")
session.use_schema('raw_pos')

# list schemas
# SQL  show schemas in database "tb_101";
schema_collection = root.databases["tb_101"].schemas.iter()
for sch in schema_collection:
  print(sch.name)

In [ ]:
#create warehouse
# create a Warehouse instance that used to store the property of a warehouse
warehouses = root.warehouses
warehouse_name = "tb_de_wh"
de_wh= Warehouse(
    name=warehouse_name,
    warehouse_size="LARGE",
    auto_suspend=60,
    initially_suspended="true",
    comment="data engg warehouse for tasty bytes",
    auto_resume="true"
)
dev_wh= Warehouse(
    name="tb_dev_wh",
    warehouse_size="XSMALL",
    auto_suspend=60,
    initially_suspended="true",
    comment="developer warehouse for tasty bytes",
    auto_resume="true"
)
# create a warehouse 

dewh  = warehouses.create(de_wh, mode='orReplace')
devwh = warehouses.create(dev_wh,mode='orReplace')

session.use_warehouse('tb_de_wh')

# list warehosues
# SQL  show warehouses like '%demo%';
warehouse_collection = warehouses.iter(like="tb%")
for wh in warehouse_collection:
  print(wh.name, wh.auto_suspend, wh.max_cluster_count)


In [ ]:
# create roles
session.use_role("securityadmin")
tbadmin_role = Role(name="tb_admin",  comment='admin for tasty bytes')
tbadmin_role = root.roles.create(tbadmin_role,  mode='ifNotExists')
tbde_role = Role(name="tb_data_engineer",  comment='data engineer for tasty bytes')
tbde_role = root.roles.create(tbde_role,  mode='ifNotExists')
tbdev_role = Role(name="tb_dev",  comment='developer for tasty bytes')
tbdev_role = root.roles.create(tbdev_role,  mode='ifNotExists')

In [ ]:
#give a grant to create hierarchey , tb_dev-> tb_data_engineer-> tb_admin -> sysadmin
# GRANT ROLE TB_ADMIN TO ROLE SYSADMIN
root.grants.grant(
  Grant(
    grantee=Grantees.role('sysadmin'), 
    securable=Securables.role('tb_admin'),
  )
)
# GRANT ROLE TB_DATA_ENGINEER TO ROLE TB_ADMIN
root.grants.grant(
  Grant(
    grantee=Grantees.role('tb_admin'), 
    securable=Securables.role('tb_data_engineer'),
  )
)
# GRANT ROLE TB_DEV TO ROLE TB_DATA ENGINEER
root.grants.grant(
  Grant(
    grantee=Grantees.role('tb_data_engineer'), 
    securable=Securables.role('tb_dev'),
  )
)


In [ ]:
# usages grants
# GRANT USAGE ON DATABASE TB_101 TO ROLE TB_DEV

root.grants.grant(
  Grant(
    grantee=Grantees.role('tb_dev'),
    securable=Securables.database('tb_101'),
    privileges=[Privileges.usage]
  )
)





In [ ]:
#create stage
#CREATE STAGE IF NOT EXISTS  tb_101.public.s3load_csv  URL = 's3://sfquickstarts/frostbyte_tastybytes/'      
session.use_role("sysadmin")
s3_stage_csv = Stage(
  name="s3load_csv", url='s3://sfquickstarts/frostbyte_tastybytes/'
)
stages = root.databases["tb_101"].schemas["public"].stages
stages.create(s3_stage_csv, mode='ifNotExists')

# this stage to use to store python procedure code so it can execute later in task
int_stage = Stage(name="mycode")
stages.create(int_stage, mode='ifNotExists')


In [ ]:
# List files in the stage
# LIST @TB_101.PUBLIC.S3LOAD_CSV;
s3files = root.databases["tb_101"].schemas["public"].stages["s3load_csv"].list_files()
for stageFile in s3files:
  print(stageFile)


In [ ]:
# define and create table and load data from above stage
shift_sales = Table(
  name="shift_sales",
  columns=[TableColumn(name="location_id", datatype="number(38,0)"),
           TableColumn(name="city", datatype="string"),
           TableColumn(name="date", datatype="date"),
           TableColumn(name="shift_sales", datatype="float"),
           TableColumn(name="shift", datatype="string"),
           TableColumn(name="month", datatype="number(2,0)"),
           TableColumn(name="day_of_week", datatype="number(2,0)"),
           TableColumn(name="city_population", datatype="number(38,0)"),       
           ]
)
root.databases["tb_101"].schemas["raw_pos"].tables.create(shift_sales,mode='orReplace')


shift_sales_schema = StructType([
StructField("location_id",StringType()),
StructField("city",StringType()),
StructField("date",StringType()),
StructField("shift_sales",StringType()),
StructField("shift",StringType()),
StructField("month",StringType()),
StructField("day_of_week", StringType()),
StructField("city_population", StringType()),

])
shift_sales_df=session.read.options({"field_delimiter": ",", "skip_header": 0}).schema(shift_sales_schema).csv("@tb_101.public.s3load_csv/analytics/shift_sales/")
copy_result = shift_sales_df.copy_into_table("shift_sales", force=True)
print(copy_result)
session.table("shift_sales").show()

In [ ]:
# show tables
tables = root.databases["tb_101"].schemas["raw_pos"].tables.iter(like="%")
for table_obj in tables:
  print(table_obj.name)

In [ ]:
# change warehouse size to xsmall

whobj=root.warehouses["tb_de_wh"].fetch()
whobj.warehouse_size="xsmall"
whobj.resource_monitor = None
root.warehouses["tb_de_wh"].create_or_update(whobj)

In [ ]:
table_ref = root.databases['tb_101'].schemas['raw_pos'].tables['shift_sales']
demo_table = table_ref.fetch()
demo_table.to_dict()

In [ ]:
# join an create view
shift_sales_df = session.table("tb_101.raw_pos.shift_sales")
shift_sales_df.show()
    

Create database from Snowflake Marketplace called "frostbyte_safegraph" and grant access to the role you are using (eg. sysadmin)

 https://app.snowflake.com/marketplace/listing/GZSNZL1CN82/safegraph-safegraph-frostbyte

In [ ]:


tb_safegraph_df=session.table('frostbyte_safegraph.public.frostbyte_tb_safegraph_s')
tb_safegraph_df.count();

tb_safegraph_df.show(10)

In [ ]:
#create a view in analytics schema

shift_sales_harmoized_df = shift_sales_df.join(tb_safegraph_df, shift_sales_df.col("LOCATION_ID") == tb_safegraph_df.col("LOCATION_ID"))\
        .select(shift_sales_df["LOCATION_ID"].as_("LOCATION_ID"),
                shift_sales_df["CITY"].as_("CITY"),
                shift_sales_df["DATE"],
                shift_sales_df["SHIFT_SALES"],
                shift_sales_df["SHIFT"],
                shift_sales_df["MONTH"],
                shift_sales_df["DAY_OF_WEEK"],
                shift_sales_df["CITY_POPULATION"],
                tb_safegraph_df["LATITUDE"],
                tb_safegraph_df["LONGITUDE"]
        )

shift_sales_harmoized_df.show()
shift_sales_harmoized_df.create_or_replace_view("TB_101.ANALYTICS.SHIFT_SALES_VW")
            


In [ ]:
ss_df = session.table("tb_101.analytics.shift_sales_vw")
ss_df.show()

In [ ]:
#create stage to load data from  parquet files
s3_stage_parquet = Stage(
  name="s3load_parquet", url='s3://sfquickstarts/data-engineering-with-snowpark-python/'
)
stages_parquet = root.databases["tb_101"].schemas["public"].stages
stages_parquet.create(s3_stage_parquet, mode='orReplace')

# List files in the parquet stage
# LIST @TB_101.PUBLIC.s3load_parquet;
s3files = root.databases["tb_101"].schemas["public"].stages["s3load_parquet"].list_files(pattern=".*truck.*")
for stageFile in s3files:
  print(stageFile)



In [ ]:
# craete table truck using schema detection
stage_loc= "@tb_101.public.s3load_parquet/pos/truck/"
truckdf = session.read.option("compression", "snappy").parquet(stage_loc)
tref=truckdf.copy_into_table("truck")
print(tref)
table_ref = root.databases['tb_101'].schemas['raw_pos'].tables['truck']
truck_table = table_ref.fetch()
truck_table.to_dict()

In [ ]:
truck_df = session.table("tb_101.raw_pos.truck")

print(truck_df.count())
shift_sales_df.show()

In [ ]:
@sproc(name='tb_101.raw_pos.ingest_data_sproc', 
       packages=['snowflake-snowpark-python'], 
       is_permanent=True, 
       replace=True,
       stage_location='@tb_101.public.mycode', 
       session=session)
def ingest_data_sproc(session: Session) -> T.Variant:
       shift_sales_schema = StructType([
              StructField("location_id",StringType()),
              StructField("city",StringType()),
              StructField("date",StringType()),
              StructField("shift_sales",StringType()),
              StructField("shift",StringType()),
              StructField("month",StringType()),
              StructField("day_of_week", StringType()),
              StructField("city_population", StringType()),

       ])
       shift_sales_df=session.read.options({"field_delimiter": ",", "skip_header": 0}).schema(shift_sales_schema).csv("@tb_101.public.s3load_csv/analytics/shift_sales/")
       copy_result = shift_sales_df.copy_into_table("shift_sales", force=True)
       stage_loc= "@tb_101.public.s3load_parquet/pos/truck/"
       truckdf = session.read.option("compression", "snappy").parquet(stage_loc)
       tref=truckdf.copy_into_table("truck")
       return ("Data loading complete!")



In [ ]:
@sproc(name='tb_101.raw_pos.change_wh_size', 
       packages=['snowflake-snowpark-python','snowflake.core'], 
       is_permanent=True, 
       replace=True,
       stage_location='@tb_101.public.mycode', 
       session=session)
def change_wh_size(session: Session, whname: str, whsize: str) -> T.Variant:
       root = Root(session) 
       whobj=root.warehouses[whname].fetch()
       whobj.warehouse_size=whsize
       whobj.resource_monitor = None
       root.warehouses[whname].create_or_update(whobj)
       return ("Warehouse Size changed")



In [ ]:
# create a Task instance in the client side
#serverless task
increase_whsize_task = Task(
    "increase_wh_size",
    definition=StoredProcedureCall(change_wh_size, args=["tb_de_wh","xlarge"], stage_location="@tb_101.public.mycode", packages=["snowflake-snowpark-python","snowflake.core"]),    
    warehouse="tb_de_wh",
    schedule=timedelta(hours=1)
)
ingest_task = Task(
    "loaddata_task",
    definition=StoredProcedureCall(ingest_data_sproc, stage_location="@tb_101.public.mycode", packages=["snowflake-snowpark-python"]),
    warehouse="tb_de_wh"
)
reduce_whsize_task = Task(
    "reduce_wh_size",
    definition="call tb_101.raw_pos.change_wh_size('tb_de_wh','xsmall')" ,
    user_task_managed_initial_warehouse_size="xsmall"
)
# create chain of task/DAG
reduce_whsize_task.predecessors = [ingest_task.name]
ingest_task.predecessors = [increase_whsize_task.name]

In [ ]:
tasks = root.databases["tb_101"].schemas["public"].tasks
task0 = tasks.create(increase_whsize_task, mode="orreplace")
task1 = tasks.create(ingest_task, mode="orreplace")
task2 = tasks.create(reduce_whsize_task, mode="orreplace")
for t in tasks.iter(like="%"):
    print(f"Definition of {t.name}: \n",  t.definition, sep="", end="\n--------------------------\n")

In [ ]:
#task2.resume()
#task1.resume()
#task0.resume()
task0.execute()

In [ ]:
task0.get_current_graphs()

In [ ]:
task0.suspend()

In [ ]:
#. OR Create a DAG

dag_name = "data_ingestion_dag" 
dag = DAG(dag_name, schedule=timedelta(hours=1))
with dag:
    dag_task1 = DAGTask("increase_wh_size",
           definition=StoredProcedureCall(change_wh_size, args=["tb_de_wh","xlarge"], stage_location="@tb_101.public.mycode", 
                    packages=["snowflake-snowpark-python","snowflake.core"]),  warehouse="tb_de_wh")    
    dag_task2 = DAGTask("loaddata_task", StoredProcedureCall(ingest_data_sproc, stage_location="@tb_101.public.mycode", packages=["snowflake-snowpark-python"]), warehouse="tb_de_wh")
    dag_task3 = DAGTask("reduce_wh_size",definition=StoredProcedureCall(change_wh_size, args=["tb_de_wh","xsmall"], stage_location="@tb_101.public.mycode", 
                    packages=["snowflake-snowpark-python","snowflake.core"]),  warehouse="tb_de_wh")
    dag_task1 >> dag_task2 >> dag_task3  # task1 is a predecessor of task2 which is predecssor of task3
pubschema = root.databases["tb_101"].schemas["public"]
dag_op = DAGOperation(pubschema)
dag_op.deploy(dag, mode="orreplace")  

dagiter = dag_op.iter_dags(like='%')
for dag_name in dagiter:
    print(dag_name)

In [ ]:
dag_op.run(dag)

In [ ]:
# data transformation and analytics
# Select
location_df = ss_df.select("date", "shift", "shift_sales", "location_id", "city")

# Filter
location_df = location_df.filter(F.col("location_id") == 1135)

# Sort
location_df = location_df.order_by(["date", "shift"], ascending=[0, 0])

# Display
location_df.show(n=20)

In [ ]:
#shift sales table
print(location_df.count())
location_df.explain()

In [ ]:
# Group by city and average shift sales
analysis_df = ss_df.group_by("city").agg(F.mean("shift_sales").alias("avg_shift_sales"))

# Sort by average shift sales
analysis_df = analysis_df.sort("avg_shift_sales", ascending=True)

# Pull to pandas and plot
analysis_df.to_pandas().plot.barh(x="CITY", y="AVG_SHIFT_SALES")

In [ ]:
# Filter to Vancouver
analysis_df = ss_df.filter(F.col("city") == "Vancouver")

# Group by location and average shift sales
analysis_df = analysis_df.group_by("location_id").agg(F.mean("shift_sales").alias("avg_shift_sales"))

# Get the location count
print("Vancouver location count:", analysis_df.count())

In [ ]:
ax = analysis_df.to_pandas().hist(column="AVG_SHIFT_SALES", bins=20)


In [ ]:
#cleanup
#task2.delete()
#task1.delete()
#task0.delete()
#dag_op.delete(dag)
#tb101_db.delete()

In [ ]:
#session.close()